# Partitioned Inference — UNI_BOX_REGRESSION_PARTITIONED
#
Inference using **PRODUCTION** alias directly, with optional SAMPLE.


In [ ]:
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark import functions as F
from snowflake.ml.registry import Registry
import time
import math


In [ ]:
# Configuration
INFERENCE_SAMPLE_FRACTION = 1

# Number of rows per batch. Leave as None to disable batching.
BATCH_SIZE = 1_000_000

DATABASE = "BD_AA_DEV"
STORAGE_SCHEMA = "SC_STORAGE_BMX_PS"
MODEL_SCHEMA = "SC_MODELS_BMX"
PARTITIONED_MODEL_NAME = "UNI_BOX_REGRESSION_PARTITIONED"
PREDICTIONS_TABLE = f"{DATABASE}.{STORAGE_SCHEMA}.INFERENCE_PREDICTIONS"

SOURCE_TABLE = f"{DATABASE}.{STORAGE_SCHEMA}.INFERENCE_DATASET_CLEANED"
MODEL_FQN = f"{DATABASE}.{MODEL_SCHEMA}.{PARTITIONED_MODEL_NAME}"


In [ ]:
session = get_active_session()
session.sql(f"USE DATABASE {DATABASE}").collect()
session.sql(f"USE SCHEMA {STORAGE_SCHEMA}").collect()

print("✅ Connected to Snowflake")
print(f"   Model: {PARTITIONED_MODEL_NAME} (PRODUCTION alias)")


## Input sampling


In [ ]:
input_df = session.table(SOURCE_TABLE)

if INFERENCE_SAMPLE_FRACTION is not None and 0 < INFERENCE_SAMPLE_FRACTION < 1:
    input_df = input_df.sample(frac=INFERENCE_SAMPLE_FRACTION)
    print(f"⚠️  Using SAMPLE (Snowpark): {INFERENCE_SAMPLE_FRACTION*100:.2f}%")
else:
    print("✅ Using FULL dataset (Snowpark)")


In [ ]:
# Create temporary view of input
input_df.create_or_replace_temp_view("INFERENCE_INPUT")


## Partitioned Inference and Save to Table (single-pass, explicit PRODUCTION alias)


In [ ]:
# Get model version information (used to tag predictions)
try:
    registry = Registry(
        session=session,
        database_name=DATABASE,
        schema_name=MODEL_SCHEMA,
    )
    model_ref = registry.get_model(PARTITIONED_MODEL_NAME)
    model_version = model_ref.version("PRODUCTION")
    # ModelVersion may expose the version name under different attributes depending on library version
    model_version_name = getattr(model_version, "version", getattr(model_version, "version_name", str(model_version)))
    print(f"✅ Using model version: {model_version_name}")
except Exception as e:
    print(f"⚠️  Could not get model version: {str(e)[:200]}")
    model_version_name = "UNKNOWN"

# Create predictions table with SQL (including auto-incremental primary key)
create_predictions_table_sql = f"""
CREATE TABLE IF NOT EXISTS {PREDICTIONS_TABLE} (
    PREDICTION_ID NUMBER(38,0) AUTOINCREMENT PRIMARY KEY,
    CUSTOMER_ID VARCHAR,
    STATS_NTILE_GROUP VARCHAR,
    WEEK VARCHAR,
    BRAND_PRES_RET VARCHAR,
    PROD_KEY VARCHAR,
    PREDICTED_UNI_BOX_WEEK FLOAT,
    MODEL_VERSION VARCHAR,
    PREDICTION_TIMESTAMP TIMESTAMP_NTZ DEFAULT CURRENT_TIMESTAMP()
)
"""

session.sql(create_predictions_table_sql).collect()
print(f"✅ Predictions table ready: {PREDICTIONS_TABLE} (with auto-incremental PRIMARY KEY)")


In [ ]:
# Truncate predictions table before running inference
session.sql(f"TRUNCATE TABLE {PREDICTIONS_TABLE}").collect()
print(f"🧹 Predictions table truncated: {PREDICTIONS_TABLE}")


In [ ]:
# Run partitioned inference and insert predictions into table, optionally in batches
print("🚀 RUNNING PARTITIONED INFERENCE AND SAVING TO TABLE (MODEL(..., PRODUCTION))")
start_time = time.time()

total_rows = input_df.count()
print(f"📊 Input rows (after sampling if applied): {total_rows:,}")

# Same ORDER BY used for batching (keyset pagination) and for ROW_NUMBER in single-pass
ORDER_COLS = "CUSTOMER_ID, WEEK, BRAND_PRES_RET, PROD_KEY"

if BATCH_SIZE is None:
    # Inference without batching (single pass)
    insert_predictions_sql = f"""
    INSERT INTO {PREDICTIONS_TABLE}
    (CUSTOMER_ID, STATS_NTILE_GROUP, WEEK, BRAND_PRES_RET, PROD_KEY, PREDICTED_UNI_BOX_WEEK, MODEL_VERSION)
    SELECT
        p.CUSTOMER_ID,
        p.STATS_NTILE_GROUP,
        p.WEEK,
        p.BRAND_PRES_RET,
        p.PROD_KEY,
        p.predicted_uni_box_week,
        '{model_version_name}' AS MODEL_VERSION
    FROM INFERENCE_INPUT t,
    TABLE(
      MODEL({MODEL_FQN}, PRODUCTION)!PREDICT(
        t.CUSTOMER_ID,
        t.STATS_NTILE_GROUP,
        t.WEEK,
        t.BRAND_PRES_RET,
        t.PROD_KEY,
        t.SUM_PAST_12_WEEKS,
        t.AVG_PAST_12_WEEKS,
        t.MAX_PAST_24_WEEKS,
        t.SUM_PAST_24_WEEKS,
        t.WEEK_OF_YEAR,
        t.AVG_AVG_DAILY_ALL_HOURS,
        t.SUM_P4W,
        t.AVG_PAST_24_WEEKS,
        t.PHARM_SUPER_CONV,
        t.WINES_LIQUOR,
        t.GROCERIES,
        t.MAX_PREV2,
        t.AVG_PREV2,
        t.MAX_PREV3,
        t.AVG_PREV3,
        t.W_M1_TOTAL,
        t.W_M2_TOTAL,
        t.W_M3_TOTAL,
        t.W_M4_TOTAL,
        t.SPEC_FOODS,
        t.NUM_COOLERS,
        t.NUM_DOORS,
        t.MAX_PAST_4_WEEKS,
        t.SUM_PAST_4_WEEKS,
        t.AVG_PAST_4_WEEKS,
        t.MAX_PAST_12_WEEKS
      ) OVER (PARTITION BY t.STATS_NTILE_GROUP)
    ) p
    """

    session.sql(insert_predictions_sql).collect()
else:
    # Inference in batches: each batch reads ONLY BATCH_SIZE rows via keyset pagination,
    # so Snowflake never materializes the full table (avoids OOM with INFERENCE_SAMPLE_FRACTION=1).
    num_batches = math.ceil(total_rows / BATCH_SIZE) if total_rows > 0 else 0
    print(f"📦 Running in batches of {BATCH_SIZE:,} rows ({num_batches} batch(es)) [keyset pagination]")

    # Keyset: (CUSTOMER_ID, WEEK, BRAND_PRES_RET, PROD_KEY) for "next page"
    last_customer_id = None
    last_week = None
    last_brand_pres_ret = None
    last_prod_key = None

    for batch_idx in range(num_batches):
        print(f"   ➜ Batch {batch_idx + 1}/{num_batches}")

        # Build predicate so we only read this batch's rows (no full-table ROW_NUMBER)
        if batch_idx == 0:
            batch_filter = "TRUE"
        else:
            # (CUSTOMER_ID, WEEK, BRAND_PRES_RET, PROD_KEY) > (last, ...) for deterministic next page
            parts = [
                "CUSTOMER_ID > COALESCE(NULLIF('{}', ''), '')".format((last_customer_id or "").replace("'", "''")),
                "(CUSTOMER_ID = '{}' AND WEEK > '{}')".format((last_customer_id or "").replace("'", "''"), (last_week or "").replace("'", "''")),
                "(CUSTOMER_ID = '{}' AND WEEK = '{}' AND BRAND_PRES_RET > '{}')".format((last_customer_id or "").replace("'", "''"), (last_week or "").replace("'", "''"), (last_brand_pres_ret or "").replace("'", "''")),
                "(CUSTOMER_ID = '{}' AND WEEK = '{}' AND BRAND_PRES_RET = '{}' AND PROD_KEY > '{}')".format((last_customer_id or "").replace("'", "''"), (last_week or "").replace("'", "''"), (last_brand_pres_ret or "").replace("'", "''"), (last_prod_key or "").replace("'", "''")),
            ]
            batch_filter = "(" + " OR ".join(parts) + ")"

        insert_predictions_sql_batch = f"""
        INSERT INTO {PREDICTIONS_TABLE}
        (CUSTOMER_ID, STATS_NTILE_GROUP, WEEK, BRAND_PRES_RET, PROD_KEY, PREDICTED_UNI_BOX_WEEK, MODEL_VERSION)
        WITH BATCH_PAGE AS (
            SELECT *
            FROM INFERENCE_INPUT
            WHERE {batch_filter}
            ORDER BY {ORDER_COLS}
            LIMIT {BATCH_SIZE}
        )
        SELECT
            p.CUSTOMER_ID,
            p.STATS_NTILE_GROUP,
            p.WEEK,
            p.BRAND_PRES_RET,
            p.PROD_KEY,
            p.predicted_uni_box_week,
            '{model_version_name}' AS MODEL_VERSION
        FROM BATCH_PAGE t,
        TABLE(
          MODEL({MODEL_FQN}, PRODUCTION)!PREDICT(
            t.CUSTOMER_ID,
            t.STATS_NTILE_GROUP,
            t.WEEK,
            t.BRAND_PRES_RET,
            t.PROD_KEY,
            t.SUM_PAST_12_WEEKS,
            t.AVG_PAST_12_WEEKS,
            t.MAX_PAST_24_WEEKS,
            t.SUM_PAST_24_WEEKS,
            t.WEEK_OF_YEAR,
            t.AVG_AVG_DAILY_ALL_HOURS,
            t.SUM_P4W,
            t.AVG_PAST_24_WEEKS,
            t.PHARM_SUPER_CONV,
            t.WINES_LIQUOR,
            t.GROCERIES,
            t.MAX_PREV2,
            t.AVG_PREV2,
            t.MAX_PREV3,
            t.AVG_PREV3,
            t.W_M1_TOTAL,
            t.W_M2_TOTAL,
            t.W_M3_TOTAL,
            t.W_M4_TOTAL,
            t.SPEC_FOODS,
            t.NUM_COOLERS,
            t.NUM_DOORS,
            t.MAX_PAST_4_WEEKS,
            t.SUM_PAST_4_WEEKS,
            t.AVG_PAST_4_WEEKS,
            t.MAX_PAST_12_WEEKS
          ) OVER (PARTITION BY t.STATS_NTILE_GROUP)
        ) p
        """

        session.sql(insert_predictions_sql_batch).collect()

        # Fetch last row of this batch to resume next batch (keyset)
        last_row_df = session.sql(f"""
            SELECT CUSTOMER_ID, WEEK, BRAND_PRES_RET, PROD_KEY
            FROM INFERENCE_INPUT
            WHERE {batch_filter}
            ORDER BY {ORDER_COLS}
            LIMIT {BATCH_SIZE}
            OFFSET {BATCH_SIZE - 1}
        """).collect()
        if last_row_df and len(last_row_df) > 0:
            row = last_row_df[-1]
            last_customer_id = str(row["CUSTOMER_ID"]) if row["CUSTOMER_ID"] is not None else None
            last_week = str(row["WEEK"]) if row["WEEK"] is not None else None
            last_brand_pres_ret = str(row["BRAND_PRES_RET"]) if row["BRAND_PRES_RET"] is not None else None
            last_prod_key = str(row["PROD_KEY"]) if row["PROD_KEY"] is not None else None
        else:
            break

elapsed = time.time() - start_time

predictions_df = session.table(PREDICTIONS_TABLE).filter(
    F.col("MODEL_VERSION") == model_version_name
)
predictions_count = predictions_df.count()
print(f"✅ {predictions_count:,} predictions saved to {PREDICTIONS_TABLE} in {elapsed:.2f}s (MODEL(..., PRODUCTION))")
print(f"   Model version: {model_version_name}")
predictions_df.order_by("PREDICTION_ID").show(10)


## (Optional) Sample Inference via Registry + PRODUCTION (Python/pandas)
#
Useful for debugging, quick validation or inspecting model output
directly in Python, using the same version labeled as PRODUCTION.


In [ ]:
print("\n🔍 OPTIONAL: SAMPLE PREDICTION VIA REGISTRY (PRODUCTION)")

try:
    # Create Registry pointing to the same database/schema of models
    registry = Registry(
        session=session,
        database_name=DATABASE,
        schema_name=MODEL_SCHEMA,
    )

    # Get the model and select the PRODUCTION version
    model_ref = registry.get_model(PARTITIONED_MODEL_NAME)
    model_version = model_ref.version("PRODUCTION")
    # Use force=True to skip package version validation if local environment differs
    local_model = model_version.load(force=True)

    # Take a small sample of input to avoid loading everything into memory
    sample_sp = input_df.limit(100)
    sample_pdf = sample_sp.to_pandas()

    # Execute local prediction using CustomModel (PartitionedUniBoxModel)
    sample_pred_pdf = local_model.predict(sample_pdf)

    print("✅ Sample prediction via Registry (PRODUCTION) completed")
    print(sample_pred_pdf.head())
except Exception as e:
    print(f"ℹ️  Skipping optional local prediction via Registry: {e}")
    print("   Note: This requires local installation of model dependencies (lightgbm, xgboost, etc.)")
    print("   The main SQL-based inference above works without these local dependencies.")
